In [2]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [3]:
%load_ext watermark

In [13]:
!ls

Dockerfile
LICENSE
README.md
action.json
another_query.txt
draft_bulk_import_multiple_notebooks.ipynb
elasticsearch.yml
elk_test
first_try_mapping.json
import_statements_query.txt
match_query.txt
minified.json
minified_testfile.json
test2.json
testbulk.json
testcreate.json
testnb.json
testprefix.json
testquery.txt


In [1]:
import sys

In [ ]:
testfile = "elk_test/import-one-year.ipynb"

In [8]:
%%bash 
json-minify elk_test/import-one-year.ipynb > minified_testfile.json

In [19]:
with open ("action.json", 'wb') as prefix:
    prefix.write('{"index": {"_index": "test", "_type": "type1", "_id":2}}\n')


In [20]:
%%bash
cat action.json minified_testfile.json > test2.json

In [21]:
%%bash
curl -s -XPOST localhost:9200/_bulk --data-binary @test2.json


{"took":251,"errors":false,"items":[{"index":{"_index":"test","_type":"type1","_id":"2","_version":1,"status":201}}]}

In [ ]:
#need to decide whether to do this with requests or try the elasticsearch python wrapper or what. 
#looks like kibana barely works at all with only 1 document, suddenly with 2 it starts showing more functionality. 